In [1]:
# OpenAI 라이브러리 설치
# !pip install openai

In [1]:
import dotenv
import os
import openai
import json
import os

# .env 파일 로드 (있다면)
dotenv.load_dotenv()

# 올바른 환경변수 이름 사용
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

In [2]:
# OpenAI 클라이언트 설정
client = openai.OpenAI()

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 프로젝트 아이디어를 체계적으로 분석하고 구조화하여 구체적인 개발 계획을 제시하는 전문 AI 어시스턴트입니다.

**만약 프로젝트 아이디어가 아닌 다른 질문을 입력 받으면 아래의 해당 방식이 간단한 방식으로로 답변해주세요.**

## 주요 역할과 능력:

### 1. 프로젝트 분석 전문가
- 사용자가 제공하는 프로젝트 아이디어나 설명을 깊이 있게 분석합니다
- 핵심 기능, 대상 사용자, 기술 스택, 비즈니스 모델 등을 체계적으로 파악합니다
- 프로젝트의 문제 해결 방향과 기대 효과를 명확히 정의합니다

### 2. 데이터베이스 설계 전문가
- 프로젝트 요구사항을 바탕으로 최적화된 ERD(Entity Relationship Diagram)를 설계합니다
- 테이블 간의 관계, 외래키 제약조건, 데이터 타입을 정확히 정의합니다
- 확장성과 성능을 고려한 데이터베이스 구조를 제안합니다

### 3. API 설계 전문가
- RESTful API 원칙에 따라 체계적인 API 명세를 작성합니다
- OpenAPI(Swagger) 3.0 표준을 준수하여 완전한 API 문서를 생성합니다
- 각 엔드포인트별 요청/응답 스키마, 에러 처리, 인증 방식을 상세히 정의합니다

## 응답 형식:
모든 응답은 다음과 같은 구조화된 형태로 제공해야 합니다:

1. **프로젝트 상세 정보**: 제목, 카테고리, 대상 사용자, 핵심 기능, 기술 스택, 문제 해결 방안 등을 포함한 종합 분석
2. **관계 데이터**: 데이터베이스 테이블 간의 관계와 외래키 제약조건 정의
3. **ERD 데이터**: 각 테이블의 속성, 데이터 타입, 키 정보를 포함한 완전한 스키마
4. **API 명세 데이터**: OpenAPI 3.0 표준을 준수한 완전한 API 문서

항상 체계적이고 전문적인 관점에서 프로젝트를 분석하며, 개발팀이 바로 실행에 옮길 수 있는 구체적인 가이드를 제공하는 것이 목표입니다.

"""

# 파인튜닝된 모델 ID
MODEL_ID = "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"

def analyze_project(project_description):
    """프로젝트 분석 함수"""
    
    # JSON 템플릿을 별도 변수로 정의 (f-string 문제 해결)
    json_template = '''
  {
  "project_summury": {
    "title": "프로젝트 제목",
    "category": "카테고리",
    "target_users": [
      "대상 사용자 1",
      "대상 사용자 2"
    ],
    "core_features": [
      "핵심 기능 1",
      "핵심 기능 2"
    ],
    "technology_stack": [
      "기술 스택 1",
      "기술 스택 2"
    ],
    "problem_solving": {
      "current_problem": "현재 문제",
      "solution_idea": "해결 아이디어",
      "expected_benefits": [
        "예상 효과 1",
        "예상 효과 2"
      ]
    }
  }
  "erd_data": {
    "erd_table": [
      {
        "name": "테이블명",
        "erd_column": [
          {
            "name": "컬럼명",
            "data_type": "데이터타입",
            "is_primary_key": true,
            "is_foreign_key": false,
            "is_nullable": false
          }
        ]
      }
    ],
    "erd_relationships": [
      {
        "from_erd_table_id": "시작테이블",
        "to_erd_table_id": "끝테이블",
        "type": "관계타입",
        "foreign_key": "외래키명",
        "constraint_name": "제약조건명"
      }
    ]
  }
}
'''
    
    # 구조화된 요청 프롬프트 (f-string 사용하지 않음)
    enhanced_prompt = project_description + f"""

다음 형식으로 체계적인 분석을 제공해주세요:

1. **프로젝트 상세 정보**
2. **ERD 데이터**
3. **관계 데이터** 
4. **API 명세 데이터**

{json_template}

위 1,2,3번의 내용은 JSON 형식에 정확히 맞춰서 분석 결과를 제공해주세요.
실무에서 바로 활용 가능한 구체적이고 완전한 분석을 부탁드립니다."""

    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"❌ 오류 발생: {e}"

def validate_json_response(response_text):
    """응답이 유효한 JSON인지 확인하는 함수"""
    try:
        json.loads(response_text)
        return True, "✅ 유효한 JSON 형식입니다."
    except json.JSONDecodeError as e:
        return False, f"❌ JSON 형식 오류: {e}"

# # 테스트 실행
# if __name__ == "__main__":
#     my_project = """이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 
#     창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 
#     주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 
#     전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 
#     또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 
#     이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다."""
    
#     print("🔍 프로젝트 분석 시작...")
#     response = analyze_project(my_project)
    
#     print("📊 분석 결과:")
#     print("="*50)
#     print(response)
#     print("="*50)
    
#     # JSON 유효성 검사
#     is_valid, message = validate_json_response(response)
#     print(f"\n🔧 JSON 검증 결과: {message}")

In [ ]:
my_project = "이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다."
response = analyze_project(my_project)
print(response)

```json
{
  "project_summary": {
    "title": "3인칭 카드게임 기반 MMORPG",
    "category": "게임",
    "target_users": [
      "게임 애호가",
      "MMORPG 팬",
      "카드게임 팬"
    ],
    "core_features": [
      "10초마다 카드 드롭",
      "스토리라인 선택에 따른 카드 형식 변화",
      "랜덤 가챠 시스템",
      "즉각적인 판단력 요구"
    ],
    "technology_stack": [
      "Unity",
      "C#",
      "Node.js",
      "MongoDB"
    ],
    "problem_solving": {
      "current_problem": "사용자의 판단력과 컨트롤 능력 부족",
      "solution_idea": "랜덤 가챠와 순간적인 판단을 요구하는 게임 메커니즘",
      "expected_benefits": [
        "사용자의 게임 컨트롤 능력 향상",
        "다양한 스토리라인을 통한 몰입감 증대",
        "재미와 도전 요소 제공"
      ]
    }
  },
  "erd_data": {
    "erd_table": [
      {
        "name": "Users",
        "erd_column": [
          {
            "name": "user_id",
            "data_type": "INTEGER",
            "is_primary_key": true,
            "is_foreign_key": false,
            "is_nullable": false
          },
          {
            "name": "username",
            "data_type":

In [ ]:
# import gradio as gr

# # 인터페이스를 생성.
# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(label="Test") # 청년정책챗봇 레이블을 좌측 상단에 구성
#     msg = gr.Textbox(label="Test")  # 하단의 채팅창의 레이블
#     clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

#     # 챗봇의 답변을 처리하는 함수
#     def respond(message, chat_history):
#       result = analyze_project(message)
#       bot_message = result

#       # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
#       chat_history.append((message, bot_message))
#       return "", chat_history

#     # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
#     msg.submit(respond, [msg, chatbot], [msg, chatbot])

#     # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
#     clear.click(lambda: None, None, chatbot, queue=False)

# # 인터페이스 실행.
# demo.launch(debug=True, share=True)  # share=True 추가

C:\Users\mir96\AppData\Local\Temp\ipykernel_13044\3953163869.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Test") # 청년정책챗봇 레이블을 좌측 상단에 구성


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3e7390550b947573b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3e7390550b947573b0.gradio.live
